In [92]:
!pip3 install requests
!pip3 install pyodbc
import json, requests, itertools, os, pandas as pd, numpy as np, pyodbc
from urllib.parse import urljoin

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [93]:
base_url = 'https://siu.admin.12twenty.com/api'
auth_key = 'ig5eb2bqnq5cec1hrud8y'
auth_url = base_url + '/client/generateAuthenticationToken?key=' + auth_key

bearer_token = requests.get(auth_url).text
auth_header = {'Authorization': 'Bearer ' + bearer_token}

In [94]:
students_url = base_url + '/V2/students'
students_params = {'PageSize' : '500'}
twelve_twenty_students_json = requests.get(students_url, headers=auth_header, params=students_params)
#print(json.dumps(studentsJson.json(), indent=2))

In [95]:
twelve_twenty_students_json = json.loads(twelve_twenty_students_json.text)
twelve_twenty_students_page = pd.json_normalize(twelve_twenty_students_json['Items'])
twelve_twenty_students_page.head()

,RoleId,StudentGroups,FullName,IsLgbtq,SelfIdentifiedGenderDescription,ParentEducationLevelId,ParentEducationLevelName,DoesParentHaveJD,AbaGraduateId,IsMultipleEnrollmentLinkedAccount,...,LastLoginDate,PreferredEmailAddress,PreferredConsolidatedIndustry.Name,PreferredConsolidatedJobFunction.Name,PreferredCountry.Name,UndergradMajor.Name,ConsolidatedMajor1.Name,Ethnicity1.Name,UndergradMajor2.Name,ConsolidatedMajor2.Name
0,3,None,Farhiya A Mohamud,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,None,Andrew Aaflaq,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,None,Sophia Aaflaq,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,None,Thomas Aaflaq,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,None,Ole Aarness,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
twelve_twenty_students = []
twelve_twenty_students.append(twelve_twenty_students_page)
num_pages = twelve_twenty_students_json['NumberOfPages']
for page in range(2, num_pages + 1):
    students_params = {'PageSize' : '500', 'PageNumber': page}
    twelve_twenty_students_json = json.loads(requests.get(students_url, headers=auth_header, params=students_params).text)
    twelve_twenty_students_page = pd.json_normalize(twelve_twenty_students_json['Items'])
    twelve_twenty_students.append(twelve_twenty_students_page)

In [97]:
twelve_twenty_students = pd.concat(twelve_twenty_students)
print(twelve_twenty_students.count())
twelve_twenty_students

C:\Users\SIU850258213\AppData\Local\Temp\ipykernel_6712\702677672.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  twelve_twenty_students = pd.concat(twelve_twenty_students)


RoleId                             30911
StudentGroups                          0
FullName                           30911
IsLgbtq                                0
SelfIdentifiedGenderDescription        0
                                   ...  
UndergradMajor4.Name                   1
UndergradMajor5.Name                   1
ConsolidatedMajor4.Name                1
ConsolidatedMajor5.Name                1
Phone1                                 1
Length: 219, dtype: int64


,RoleId,StudentGroups,FullName,IsLgbtq,SelfIdentifiedGenderDescription,ParentEducationLevelId,ParentEducationLevelName,DoesParentHaveJD,AbaGraduateId,IsMultipleEnrollmentLinkedAccount,...,PreferredConsolidatedJobFunction3.Name,UndergradMajor3.Name,ConsolidatedMajor3.Name,PreferredConsolidatedJobFunction4.Name,PreferredConsolidatedJobFunction5.Name,UndergradMajor4.Name,UndergradMajor5.Name,ConsolidatedMajor4.Name,ConsolidatedMajor5.Name,Phone1
0,3,None,Farhiya A Mohamud,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,None,Andrew Aaflaq,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,None,Sophia Aaflaq,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,None,Thomas Aaflaq,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,None,Ole Aarness,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,3,None,Trent Zumwalt,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,3,None,Khin Thet Zun,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,3,None,Martin Zuniga,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,3,None,Gina Zunke,None,None,None,None,None,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
print(twelve_twenty_students.columns.tolist())

columnMapping = {'Id':'12TwentyId',
                 'FirstName':'FirstName',
                 'MiddleName':'MiddleName',
                 'LastName' : 'LastName',
                 'EmailAddress' : 'EmailAddress',
                 'IsEnrolled': 'IsEnrolled',
                 'StudentId': 'StudentId',
                 'College.Name': 'College',
                 'Program.Name': 'Program',
                 'GraduationYearId' : 'GraduationYear',
                 'GraduationTerm' : 'GraduationTerm',
                 'DegreeLevel.Name' : 'DegreeLevel',
                 'CustomAttributeValues.custom_attribute_10888805132042': 'AppliedForGraduation'
}

for column in twelve_twenty_students.columns:
    if column not in columnMapping.keys():
        twelve_twenty_students.drop(column, axis=1, inplace=True)

twelve_twenty_students.rename(columns = columnMapping, inplace='True')

twelve_twenty_students.set_index('StudentId')
twelve_twenty_students


['RoleId', 'StudentGroups', 'FullName', 'IsLgbtq', 'SelfIdentifiedGenderDescription', 'ParentEducationLevelId', 'ParentEducationLevelName', 'DoesParentHaveJD', 'AbaGraduateId', 'IsMultipleEnrollmentLinkedAccount', 'ReportingCategoryMbaCseaPostGraduation', 'ReportingCategoryMbaCseaInternship', 'IsWorkStudyEligible', 'FirstName', 'LastName', 'EmailAddress', 'GraduationYearId', 'GraduationClass', 'GraduationTerm', 'StudentId', 'Id', 'IsAlumni', 'IncludeInResumeBook', 'JoinDate', 'IsEnrolled', 'IsFlagged', 'IsFirstGenerationCollegeStudent', 'LinkedInProfileUrl', 'IsTransferStudent', 'DeclineToStateIsTransferStudent', 'HasPhoto', 'AssignedAdvisor', 'AssignedAdvisor2', 'AssignedAdvisor3', 'AssignedAdvisor4', 'AssignedAdvisor5', 'SubInfoDisplay', 'OutcomeStatusInternship.Name', 'OutcomeStatusInternship.AttributeId', 'OutcomeStatusInternship.Id', 'OutcomeStatusPostGraduation.Name', 'OutcomeStatusPostGraduation.AttributeId', 'OutcomeStatusPostGraduation.Id', 'CountryOfCitizenship1.AttributeId',

,FirstName,LastName,EmailAddress,GraduationYear,GraduationTerm,StudentId,12TwentyId,IsEnrolled,College,Program,DegreeLevel,AppliedForGraduation,MiddleName
0,Farhiya,A Mohamud,farhiya.amohamud@siu.edu,2025,Spring,SIU856552554,540016053171738,True,SIU,SIU,Senior,True,NaN
1,Andrew,Aaflaq,andrew.aaflaq@siu.edu,2026,Spring,SIU854000305,540016052940723,True,SIU,SIU,Graduate Masters,False,Wasfi
2,Sophia,Aaflaq,sophia.aaflaq@siu.edu,2025,Spring,SIU856512099,540016053171557,True,SIU,SIU,Senior,True,H
3,Thomas,Aaflaq,thomas.aaflaq@siu.edu,2027,Spring,SIU856575241,540016053969384,True,SIU,SIU,Junior,False,Clement
4,Ole,Aarness,ole.aarness@siu.edu,2024,Summer,SIU850311584,540016053170730,True,SIU,SIU,Alumni,False,Kristian
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,Trent,Zumwalt,trenton.zumwalt@siu.edu,2027,Spring,SIU856553451,540016053174796,True,SIU,SIU,Junior,False,L
407,Khin Thet,Zun,khinthet.zun@siu.edu,2028,Fall,SIU856583066,540016053595691,True,SIU,SIU,CESL,False,NaN
408,Martin,Zuniga,martin.zuniga@siu.edu,2026,Spring,SIU856548211,540016053169878,False,SIU,SIU,Junior,False,NaN
409,Gina,Zunke,gina.zunke@siu.edu,2027,Spring,SIU856589188,540016054331131,True,SIU,SIU,Graduate Masters,False,Carol


In [99]:
sql_connection = pyodbc.connect("Driver={SQL Server};"
                      "Server=itapp-ssis-dev;"
                      "Database=MableySandbox;"
                      "Trusted_Connection=yes;")


In [100]:
ods_query = "SELECT * FROM TwelveTwentyStudents;"
ods_students = pd.read_sql(ods_query,sql_connection)
ods_students.insert(len(ods_students.columns), 'ActionNeeded', 'None')
ods_students.insert(len(ods_students.columns), 'Result', '')
ods_students.insert(len(ods_students.columns), 'Message', '')
ods_students.set_index('StudentId')

C:\Users\SIU850258213\AppData\Local\Temp\ipykernel_6712\2585699019.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odsStudentsDataFrame = pd.read_sql(odsQuery,sqlConnection)


,EmailAddress,FirstName,MiddleName,LastName,GraduationYear,GraduationTerm,ExpectedGraduationDate,IsEnrolled,Program,DegreeLevel,College,AppliedForGrad,SystemLabelNames,ActionNeeded,Result,Message
StudentId,,,,,,,,,,,,,,,,
SIU850000649,danielle.cox@siu.edu,Danielle,Nichole,Cox,2025,Spring 2025,2025-05-09,Yes,SIU,Senior,SIU,True,None,None,,
SIU850001358,james.w.black@siu.edu,James,W,Black,2025,Spring 2025,2025-05-09,Yes,SIU,Senior,SIU,False,None,None,,
SIU850001815,thomas.distelhurst@siu.edu,Thomas,None,Distelhurst,2024,Spring 2024,2024-05-10,Yes,SIU,Alumni,SIU,False,None,None,,
SIU850001876,mnowak@siu.edu,Matthew,G,Nowak,2024,Summer 2024,2024-08-02,Yes,SIU,Alumni,SIU,False,None,None,,
SIU850002168,vickie.walls@siu.edu,Vickie,Anna,McKinney,2025,Spring 2025,2025-05-09,Yes,SIU,Alumni,SIU,False,None,None,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIU856590916,bronson.flint@siu.edu,Bronson,None,Flint,2027,Spring 2027,2027-05-07,Yes,SIU,Graduate Masters,SIU,False,None,None,,
SIU856590927,carly.cox@siu.edu,Carly,None,Cox,2027,Spring 2027,2027-05-07,Yes,SIU,Junior,SIU,False,None,None,,
SIU856591363,cynthia.vandeventer@siu.edu,Cynthia,Cae,Vandeventer,2027,Spring 2027,2027-05-07,Yes,SIU,Graduate Masters,SIU,False,None,None,,


In [101]:
ods_students.loc[~ods_students['StudentId'].isin(twelve_twenty_students['StudentId']), 'ActionNeeded'] = 'Insert'
ods_students.loc[~ods_students['StudentId'].isin(twelve_twenty_students['StudentId'])]

,EmailAddress,FirstName,MiddleName,LastName,StudentId,GraduationYear,GraduationTerm,ExpectedGraduationDate,IsEnrolled,Program,DegreeLevel,College,AppliedForGrad,SystemLabelNames,ActionNeeded,Result,Message
21,clnteth@siu.edu,Jennifer,S,Sherry,SIU850008193,2025,Spring 2025,2025-05-09,Yes,SIU,Alumni,SIU,False,None,Insert,,
35,chris.d.leonard1@siu.edu,Chris,D,Leonard,SIU850012251,2024,Summer 2024,2024-08-02,Yes,SIU,Alumni,SIU,False,None,Insert,,
83,teke4@siu.edu,Tequila,Anne,Young,SIU850033580,2023,Summer 2023,2023-08-04,Yes,SIU,Alumni,SIU,False,None,Insert,,
98,ksummers@siu.edu,Kamden,S,Summers,SIU850039493,2027,Spring 2027,2027-05-07,Yes,SIU,Graduate Masters,SIU,False,None,Insert,,
107,clarence.davis@siu.edu,Clay,A,Davis,SIU850043070,2024,Spring 2024,2024-05-10,Yes,SIU,Alumni,SIU,False,None,Insert,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9501,noah.schroeder@siu.edu,Noah,None,Schroeder,SIU856532174,2024,Spring 2024,2024-05-10,Yes,SIU,Alumni,SIU,False,None,Insert,,
9533,baila.moller@siu.edu,Baila,None,Moller,SIU856532619,2025,Spring 2025,2025-05-09,Yes,SIU,Alumni,SIU,False,None,Insert,,
9567,julia.white@siu.edu,Julia,Ann,White,SIU856533034,2025,Spring 2025,2025-05-09,Yes,SIU,Alumni,SIU,False,None,Insert,,
9924,charnita.little@siu.edu,Charnita,Renee,Little,SIU856537083,2025,Spring 2025,2025-05-09,Yes,SIU,Alumni,SIU,False,None,Insert,,


In [102]:
# idx = odsStudentsDataFrame.index.intersection(allStudentData)
# odsStudentsDataFrame.loc[idx].eq(allStudentData.loc[idx])
# studentsToUpdate = odsStudentsDataFrame.where(odsStudentsDataFrame['StudentId'] == allStudentData['StudentId'])

merged_students = pd.merge(ods_students, twelve_twenty_students, on ='StudentId', how='left', suffixes=['_ODS', '_12Twenty'])
merged_students

,EmailAddress_ODS,FirstName_ODS,MiddleName_ODS,LastName_ODS,StudentId,GraduationYear_ODS,GraduationTerm_ODS,ExpectedGraduationDate,IsEnrolled_ODS,Program_ODS,...,EmailAddress_12Twenty,GraduationYear_12Twenty,GraduationTerm_12Twenty,12TwentyId,IsEnrolled_12Twenty,College_12Twenty,Program_12Twenty,DegreeLevel_12Twenty,AppliedForGraduation,MiddleName_12Twenty
0,danielle.cox@siu.edu,Danielle,Nichole,Cox,SIU850000649,2025,Spring 2025,2025-05-09,Yes,SIU,...,danielle.cox@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Senior,True,Nichole
1,james.w.black@siu.edu,James,W,Black,SIU850001358,2025,Spring 2025,2025-05-09,Yes,SIU,...,james.w.black@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Senior,False,W
2,thomas.distelhurst@siu.edu,Thomas,None,Distelhurst,SIU850001815,2024,Spring 2024,2024-05-10,Yes,SIU,...,thomas.distelhurst@siu.edu,2024.0,Spring,5.400161e+14,True,College of Arts and Media,SIU,Alumni,False,NaN
3,mnowak@siu.edu,Matthew,G,Nowak,SIU850001876,2024,Summer 2024,2024-08-02,Yes,SIU,...,mnowak@siu.edu,2024.0,Summer,5.400161e+14,True,College of Liberal Arts,SIU,Alumni,False,G
4,vickie.walls@siu.edu,Vickie,Anna,McKinney,SIU850002168,2025,Spring 2025,2025-05-09,Yes,SIU,...,vickie.walls@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Alumni,False,Anna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15518,bronson.flint@siu.edu,Bronson,None,Flint,SIU856590916,2027,Spring 2027,2027-05-07,Yes,SIU,...,bronson.flint@siu.edu,2027.0,Spring,5.400161e+14,True,SIU,SIU,Graduate Masters,False,NaN
15519,carly.cox@siu.edu,Carly,None,Cox,SIU856590927,2027,Spring 2027,2027-05-07,Yes,SIU,...,carly.cox@siu.edu,2027.0,Spring,5.400161e+14,True,SIU,SIU,Junior,False,NaN
15520,cynthia.vandeventer@siu.edu,Cynthia,Cae,Vandeventer,SIU856591363,2027,Spring 2027,2027-05-07,Yes,SIU,...,cynthia.vandeventer@siu.edu,2027.0,Spring,5.400161e+14,True,SIU,SIU,Graduate Masters,False,Cae
15521,tony.mongulubodiko@siu.edu,Tony,None,Mongulu Bodiko,SIU856592706,2027,Spring 2027,2027-05-07,Yes,SIU,...,tony.mongulubodiko@siu.edu,2027.0,Spring,5.400161e+14,True,SIU,SIU,Graduate Masters,False,NaN
